<a href="https://colab.research.google.com/github/hanumantjain/agentic-ai/blob/main/AgentLoopWithFunctionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!!pip install litellm --quiet

import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [6]:
import json
import json
from typing import List
from litellm import completion

def list_files() -> List[str]:
  return os.listdir('.')

def read_file(file_name: str) -> str:
  try:
    with open(file_name, 'r') as file:
      return file.read()
  except FileNotFoundError:
    return f"Error: {file_name} not found."
  except Exception as e:
    return f"Error: {str(e)}"

def terminate(message: str) -> None:
  print(f"Termination message: {message}")

In [7]:
tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate
}

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": { "type": "object", "properties": {}, "required": [] }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory",
            "parameters": {
                "type": "object",
                "properties": { "file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"}
                },
                "required": ["message"]
            }
        }
    }
]

In [9]:
agent_rules = [{
    'role': "system",
    "content": """
        You are an AI agent that can perform tasks by using available tools.

        If a user asks about files, documents or content, first list the files before reading them.

        When you are done, terminate the conversation by using the "terminate" tool and provide the results to the user.
    """
}]

In [12]:
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory =[{
    "role": "user",
    "content": user_task
}]

while iterations < max_iterations:
  messages = agent_rules + memory

  response = completion(
      model="openai/gpt-4o-mini",
      messages = messages,
      tools= tools,
      max_tokens = 512
  )
  if response.choices[0].message.tool_calls:
    tool = response.choices[0].message.tool_calls[0]
    tool_name = tool.function.name
    tool_args = json.loads(tool.function.arguments)

    action = {
        "tool_name": tool_name,
        "args": tool_args
    }

    if tool_name == "terminate":
      print(f"Termination message: {tool_args['message']}")
    elif tool_name in tool_functions:
      try:
        result = {"result": tool_functions[tool_name](**tool_args)}
      except Exception as e:
        result = {"error": f"Error executing {tool_name}"}
    else:
      result = {"error": f"Unknow tool: {tool_name}"}

    print(f"Exceuting: {tool_name} with args {tool_args}")
    print(f"Result: {result}")
    memory.extend([
        {
            "role": "assistant", "content": json.dumps(action)
        },
        {
            "role": "user", "content":json.dumps(result)
        }
    ])
  else:
    result = response.choices[0].message.content
    print(f"Response: {result}")
    break


What would you like me to do? read each of the files in the current directory and tell me what they do
Exceuting: list_files with args {}
Result: {'result': ['.config', 'sample_data']}
Exceuting: read_file with args {'file_name': '.config'}
Result: {'result': "Error: [Errno 21] Is a directory: '.config'"}
Exceuting: read_file with args {'file_name': 'sample_data'}
Result: {'result': "Error: [Errno 21] Is a directory: 'sample_data'"}
Response: It seems that both ".config" and "sample_data" are directories, not files. There are no readable files in the current directory.

I will now terminate the conversation. If you need any further assistance, feel free to ask! 

terminate({"message": "No readable files found in the current directory."})
